In [2]:
import openai
import requests
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from amadeus import Client, ResponseError
import pandas as pd

In [3]:
## This Cell is Used For Authentication and Token Retrieval ##

# API Authentication
auth_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': 'RW5x5Bn2aMZYmgyHCyomxPetDteMmM2a',
    'client_secret': 'sKvaa9jS36eO7OCL'
}

# Get access token
auth_response = requests.post(auth_url, data=auth_data)
access_token = auth_response.json()['access_token']

# Use token in API request
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [28]:
## This Cell Pulls the Requested Flight Data and Converts it Into a Pandas Data Frame ##

#Pull the Flight List API
flight_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
params= {
    'originLocationCode': 'CLE',  # Cleveland
    'destinationLocationCode': 'NYC',  # New York
    'departureDate': '2025-05-01',
    'adults': 1,
    'nonStop': 'false',
    'currencyCode': 'USD',
    'max': 10  # Limit results
}

respone_flight = requests.get(flight_url, headers=headers, params=params)
flight_data = respone_flight.json()

# Return requested data and convert to a pandas data frame
flights_list = []
for offer in flight_data['data']:
        # Extract the most important data from each flight offer
        flight_info = {
            'offer_id': offer.get('id', 'ID not available'),
            'price': offer.get('price', {}).get('total', 'Price not available'),
            'currency': offer.get('price', {}).get('currency', 'Currency not available'),
            # Get the first itinerary segment information (typically the outbound flight)
            'departure_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('iataCode', 'Airport not available'),
            'arrival_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[-1].get('arrival', {}).get('iataCode', 'Airport not available'),
            'departure_date': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('at', 'Date not available'),
            'airline': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('carrierCode', 'Airline not available')
        }
        flights_list.append(flight_info)
flight_df = pd.DataFrame(flights_list)

# Convert departure date to readable format
flight_df['departure_date'] = pd.to_datetime(flight_df['departure_date']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Format price to display as currency
flight_df['price'] = flight_df['price'].apply(lambda x: f"${float(x):,.2f}")

# Fill missing values with placeholder text
flight_df.fillna("Not available", inplace=True)

# Sort by price (ascending)
flight_df = flight_df.sort_values(by='price', ascending=True)

# Print the DataFrame
print(flight_df)


  offer_id    price currency departure_airport arrival_airport  \
0        1  $116.99      USD               CLE             LGA   
1        2  $116.99      USD               CLE             JFK   
2        3  $116.99      USD               CLE             JFK   
3        4  $124.47      USD               CLE             EWR   
4        5  $134.17      USD               CLE             EWR   
5        6  $154.47      USD               CLE             EWR   
6        7  $216.45      USD               CLE             EWR   
7        8  $226.15      USD               CLE             LGA   
8        9  $226.15      USD               CLE             LGA   
9       10  $226.15      USD               CLE             LGA   

        departure_date airline  
0  2025-05-01 10:15:00      B6  
1  2025-05-01 10:15:00      B6  
2  2025-05-01 10:15:00      B6  
3  2025-05-01 07:00:00      UA  
4  2025-05-01 06:00:00      UA  
5  2025-05-01 08:40:00      UA  
6  2025-05-01 19:20:00      UA  
7  2025-0

In [30]:
## This Cell Pulls Requested Hotel Information and Converts it Into a Pandas Data Frame ##

#Pulls the Hotels API
hotels_url = 'https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city'
params= {
    'cityCode': 'NYC',  # IATA code for New York
    'radius': 5,
    'radiusUnit': 'KM',
    'ratings': '3,4,5',  # Optional: filter by star ratings
    'hotelSource': 'ALL'  # ALL, BEDBANK, or DIRECTCHAIN
}

response_hotel = requests.get(hotels_url, headers=headers, params=params)
hotel_data = response_hotel.json()

# Return requested data and convert to a pandas data frame
hotels_list = []
for hotel in hotel_data['data']:
    hotel_info = {
        'name': hotel.get('name', 'Name not available'),
        'hotel_id': hotel.get('hotelId', 'ID not available'),
        'rating': hotel.get('rating', 'Rating not available'),
    }
    hotels_list.append(hotel_info)
hotels_df = pd.DataFrame(hotels_list)
print(hotels_df)

                              name  hotel_id  rating
0               DUANE STREET HOTEL  XTNYC130       4
1    HAMPTON INN MANHATTAN SEAPORT  HXNYC320       3
2         SMYTH - A THOMPSON HOTEL  TMNYC822       4
3     COSMOPOLITAN HOTEL - TRIBECA  LENYC7A3       4
4      HOLIDAY INN NYC WALL STREET  HINYC596       5
..                             ...       ...     ...
128                      ROW HOTEL  XTNYCMIL       4
129    THE MUSE NY A KIMPTON HOTEL  KCNYCMSE       5
130        W NEW YORK TIMES SQUARE  WHNYC234       5
131  BEST WESTERN PLUS HOSPITALITY  BWNYC114       3
132       WALDORF ASTORIA NEW YORK  WANYC506       4

[133 rows x 3 columns]


In [31]:
## This Cell Pulls Tours and Activities Information for New York City ##

# Pull the Tours and Activities API
tours_url = 'https://test.api.amadeus.com/v1/shopping/activities'
params = {
    'latitude': 40.712776,   # New York City latitude
    'longitude': -74.005974, # New York City longitude
    'radius': 20,            # 20km radius
    'currencyCode': 'USD'    # Return prices in USD
}
response_tours = requests.get(tours_url, headers=headers, params=params)
tours_data = response_tours.json()

# Return requested data and convert to a pandas data frame
activities_list = []
for activity in tours_data.get('data', []):
    # Get price and booking_link values
    price = activity.get('price', {}).get('amount')
    booking_link = activity.get('bookingLink')
    
    # Only add activities where both price and booking_link are available
    if price is not None and booking_link is not None and price != 'Price not available' and booking_link != 'Link not available':
        activity_info = {
            'name': activity.get('name', 'Name not available'),
            'price': price,
            'booking_link': booking_link
        }
        activities_list.append(activity_info)
            
activities_df = pd.DataFrame(activities_list)
print(activities_df)

                                                 name   price  \
0     Bourbon Basics Whiskey Tasting // West Village     60.0   
1   A Tour of Italy: Cooking Class // Greenwich Vi...   162.0   
2   Afternoon Plant-Based Lebanese Cooking Class &...    78.0   
3   Bartender's Signature Cocktail Class // West V...   120.0   
4        Pasta Workshop & Dinner // Greenwich Village   162.0   
5   Pasta with Homemade Sauce Cooking Class & Mult...    90.0   
6    Bread & Pastry Making Class // Greenwich Village   162.0   
7   Plant-Based Lebanese Cooking Class // Greenwic...    78.0   
8         A Man and his Sandwich Tour // East Village    72.0   
9                 Viva Esapana - make your own tapas    160.8   
10  Explore Manhattan from above in a spectacular ...   320.0   
11                           Vol découverte de Mâcon      0.0   
12                                 NY helicopter tour   330.0   
13    KHAN-TOKE Modern Northern Thai Lunch // Midtown    48.0   
14           Multi-Course

In [32]:
# Sample functions to call APIs
def get_flights(origin, destination, departure_date):
    # Sample Amadeus API call
    flight_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {
        'originLocationCode': origin,
        'destinationLocationCode': destination,
        'departureDate': departure_date,
        'adults': 1,
        'nonStop': 'false',
        'currencyCode': 'USD',
        'max': 5
    }
    response = requests.get(flight_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Unable to fetch flight data"}

def chatbot_response(user_input):
    if "find flight" in user_input.lower():
        return get_flights("CLE", "NYC", "2025-05-01")
    else:
        return "Sorry, I didn’t understand that."

# Simulate a conversation
user_input = "find flight"
print("User:", user_input)
bot_response = chatbot_response(user_input)
print("Bot:", bot_response)


User: find flight
Bot: {'meta': {'count': 5, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=CLE&destinationLocationCode=NYC&departureDate=2025-05-01&adults=1&nonStop=false&currencyCode=USD&max=5'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-04-04', 'lastTicketingDateTime': '2025-04-04', 'numberOfBookableSeats': 7, 'itineraries': [{'duration': 'PT5H19M', 'segments': [{'departure': {'iataCode': 'CLE', 'at': '2025-05-01T10:15:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'C', 'at': '2025-05-01T11:57:00'}, 'carrierCode': 'B6', 'number': '2040', 'aircraft': {'code': '223'}, 'operating': {'carrierCode': 'B6'}, 'duration': 'PT1H42M', 'id': '3', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'BOS', 'terminal': 'C', 'at': '2025-05-01T14:05:00'}, 'arrival': {'iataCode': 'LGA'